In [1]:
# Create the final SUMS candidate catalog
# - I checked for duplicates in topcat but there were none 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import os
data_dir = os.getenv("DATADIR")


# Read in the candidate catalog 
df = pd.read_csv(data_dir + "0_SUMS_Catalogs/CandidateCatalog/3_stripped_star_candidates.csv")

# Sort by RA
df = df.sort_values(by='ra').reset_index(drop=True)

# Resort columns
cols = [ # Basic info
        'SUMS_ID', 'ra', 'dec', 'cut',
        # UV Photometry
        'uvw2', 'uvw2_err', 'uvm2', 'uvm2_err',  'uvw1', 'uvw1_err',
        # Optical Photometry
        'U', 'e_U', 'B', 'e_B', 'V', 'e_V', 'I', 'e_I',
        # Standard Deviation 
        'uvw2_std', 'uvm2_std', 'uvw1_std',
        # Flux Frac 
        'uvw2_flux_frac', 'uvm2_flux_frac', 'uvw1_flux_frac',
        # Residual Frac
        'uvw2_resid_frac',  'uvm2_resid_frac', 'uvw1_resid_frac',
        # Dist Moved
        'uvw2_dist_moved',  'uvm2_dist_moved', 'uvw1_dist_moved',
        # Dist to nearest neighbor 
        'uvw2_closest_min',  'uvm2_closest_min', 'uvw1_closest_min',
        # Num5 
        'uvw2_num5','uvm2_num5', 'uvw1_num5', 
        # Num2.5
        'uvw2_num2p5',  'uvm2_num2p5', 'uvw1_num2p5',
        # N observations 
        'uvw2_num_obs', 'uvm2_num_obs',  'uvw1_num_obs',
        # Gaia Chi2
        'gaia_chi2']
df = df[cols]

# Rename Columns
df.columns = ['SUMS_ID', 'RA', 'Dec', 'Ranking',
              
              'UVW2', 'UVW2_err', 'UVM2', 'UVM2_err', 'UVW1', 'UVW1_err',

              'U', 'U_err', 'B', 'B_err', 'V', 'V_err', 'I', 'I_err',

              'UVW2_std', 'UVM2_std', 'UVW1_std',

              'UVW2_flux_frac', 'UVM2_flux_frac', 'UVW1_flux_frac',

              'UVW2_resid_frac', 'UVM2_resid_frac', 'UVW1_resid_frac',

              'UVW2_dist_moved', 'UVM2_dist_moved', 'UVW1_dist_moved',

              'UVW2_dist_neighbor', 'UVM2_dist_neighbor', 'UVW1_dist_neighbor',

              'UVW2_n5', 'UVM2_n5', 'UVW1_n5',

              'UVW2_n2p5', 'UVM2_n2p5', 'UVW1_n2p5',

              'UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs',
              
              'Gaia_chi2']

# Some Columns need to be integers
integer_cols = ['UVW2_n5', 'UVM2_n5', 'UVW1_n5',

              'UVW2_n2p5', 'UVM2_n2p5', 'UVW1_n2p5',

              'UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs']

for col in integer_cols:
    # Convert nans to -99 
    df[col] = df[col].fillna(-99)
    # Convert to int
    df[col] = df[col].astype(int)
    # Convert -99 back to nan
    df[col] = df[col].replace(-99, np.nan)
    
# Report same SF as MCPS
sig_fig_cols = ['UVW2', 'UVM2', 'UVW1','UVW2_err', 'UVM2_err', 'UVW1_err','UVW2_std', 'UVM2_std', 'UVW1_std',

              'UVW2_flux_frac', 'UVM2_flux_frac', 'UVW1_flux_frac',

              'UVW2_resid_frac', 'UVM2_resid_frac', 'UVW1_resid_frac',

              'UVW2_dist_moved', 'UVM2_dist_moved', 'UVW1_dist_moved',

              'UVW2_dist_neighbor', 'UVM2_dist_neighbor', 'UVW1_dist_neighbor']


for col in sig_fig_cols:
  df[col] = df[col].round(2)

df['Gaia_chi2'] = df['Gaia_chi2'].round(1)

print('Final Size:', df.shape[0])
# Save final catalog 
df.to_csv(data_dir + '0_SUMS_Catalogs/FinalCatalogs/SUMS_UV_Candidate_Catalog.csv', index=False)
df['sums_id_noedit'] = df.SUMS_ID # Later want to pull ids without the formatting for the latex table

Final Size: 820


In [2]:
# Choose a few rows that have data and are more typical
# Only show some columns otherwise it's too wide
# Add a columns of 'vdots' to split 
df['\\hdots'] = ['\\hdots']*df.shape[0]
selection = df.loc[df.sums_id_noedit.isin(['SUMS_79e24','SUMS_832c6','SUMS_8c796','SUMS_8dfe7']),['SUMS_ID', 'RA', 'Dec','Ranking', 
              
               'UVW2', 'UVW2_err', #'UVM2', 'UVM2_err', 'UVW1', 'UVW1_err',

                '\\hdots',

            #   'U', 'U_err', 'B', 'B_err', 'V', 'V_err', 'I', 'I_err',

            #   'UVW2_std', 'UVM2_std', 'UVW1_std',

            #   'UVW2_flux_frac', 'UVM2_flux_frac', 'UVW1_flux_frac',

            #   'UVW2_resid_frac', 'UVM2_resid_frac', 'UVW1_resid_frac',

            #   'UVW2_dist_moved', 'UVM2_dist_moved', 'UVW1_dist_moved',

       #        'UVW2_dist_neighbor', 'UVM2_dist_neighbor', 'UVW1_dist_neighbor',

            #   'UVW2_n5', 'UVM2_n5', 'UVW1_n5',

           #   'UVW2_n2p5', 'UVM2_n2p5', 
         #    'UVW1_n2p5',

              'UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs','Gaia_chi2']]

# Format the floats
for col in [ 'UVW2', 'UVW2_err']:
  selection[col] = selection[col].apply(lambda x: '{:.5g}'.format(x))
# Format the integers 
for col in ['UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs']:
  selection[col] = selection[col].apply(lambda x: '{:.0f}'.format(x))
selection['Gaia_chi2'] = selection['Gaia_chi2'].apply(lambda x: '{:.1f}'.format(x))

# Anything with underscores needs a \\ in front of it 
df['SUMS_ID'] = df['SUMS_ID'].apply(lambda x: x.replace('_','$\\_$'))
columns = selection.columns
columns = [col.replace('_','$\\_$') for col in columns]
selection.columns = columns


print(selection.to_latex(index=False))

\begin{tabular}{lrrllllllll}
\toprule
SUMS$\_$ID & RA & Dec & Ranking & UVW2 & UVW2$\_$err & \hdots & UVW2$\_$nobs & UVM2$\_$nobs & UVW1$\_$nobs & Gaia$\_$chi2 \\
\midrule
SUMS_8c796 & 10.414935 & -73.377410 & B-E & 18.38 & 0.05 & \hdots & 2 & 1 & 3 & 0.1 \\
SUMS_8dfe7 & 10.831395 & -73.659790 & B-G & 17.44 & 0.03 & \hdots & 2 & 2 & 2 & 1.8 \\
SUMS_79e24 & 84.238275 & -69.458760 & VB-E & 17.77 & 0.03 & \hdots & 8 & 8 & 8 & 1.2 \\
SUMS_832c6 & 85.523865 & -69.205750 & VB-G & 18.73 & 0.1 & \hdots & 1 & 2 & 2 & 5.7 \\
\bottomrule
\end{tabular}

